In [1]:
import numpy as np
from scipy.stats import gamma

Indian Buffet Process Prior
====

In [2]:
# Indian Buffet Process Function

def sampleIBP(alpha, num_objects):
    
    # Initializing storage for results
    result = np.zeros([num_objects, 1000])
    # Draw from the prior for alpha
    t = np.random.poisson(alpha)
    # Filling in first row of result matrix
    result[0, 0:t] = np.ones([1, t])
    # Initializing K+
    K_plus = t
    

    for i in range(1, num_objects):
        for j in range(0, K_plus):
            p = np.array([np.log(np.sum(result[0:i,j])) - np.log(i+1), 
                          np.log(i+1 - np.sum(result[0:i, j])) - np.log(i+1)])
            p = np.exp(p - max(p))

            if(np.random.uniform() < p[0]/np.sum(p)):
                result[i, j] = 1
            else:
                result[i, j] = 0
        t = np.random.poisson(alpha/(i+1))
        x = K_plus + 1
        y = K_plus + t
        result[i, (x-1):y] = np.ones([1, t]) # NEED TO CHECK DIM
        K_plus = K_plus+t
    result = result[:, 0:K_plus]
    return list([result, K_plus])

In [3]:
sampleIBP(1, 1)

[array([[ 1.,  1.]]), 2]

Inverse Simplification
====

In [4]:
# Function for enhancing inverse process

def calcInverse(Z, M, i, k, val):
    M_i = M - (np.dot(np.dot(np.dot(M, Z[i,:].T), Z[i,:]), M)) / (np.dot(np.dot(Z[i,:], M), Z[i,:].T) - 1)
    Z[i, k] = val
    M = M_i - (np.dot(np.dot(np.dot(M_i, Z[i,:].T), Z[i,:]), M_i)) / (np.dot(np.dot(Z[i,:], M_i), Z[i,:].T) + 1)
    Inv = M
    return(Inv)

def calcInverse1(Z, M, i, k, val):
    # Calculating M_i
    part1 = M
    part2 = np.dot(M, Z[i,:].T)
    part3 = np.dot(part2, Z[i,:])
    part4 = np.dot(part3, M)
    part5 = np.dot(Z[i,:], M)
    part6 = np.dot(part5, Z[i,:].T) - 1

    M_i = part1 - part4 / part6

    Z[i, k] = val

    #Calculating M
    part1 = M_i
    part2 = np.dot(M_i, Z[i,:].T)
    part3 = np.dot(part2, Z[i,:])
    part4 = np.dot(part3, M_i)
    part5 = np.dot(Z[i,:], M_i)
    part6 = np.dot(part5, Z[i,:].T) + 1

    M = part1 - part4 / part6

    Inv = M
    return(Inv)

Likelihood Function 
====

In [5]:
## Define the likelihood function ##
# The function returns the log likelihood
def likelihood(X, Z, M, sigma_A, sigma_X, K_plus, num_objects, object_dim):
    part1 = - (num_objects*(object_dim/2)*np.log(2*np.pi))
    part2 = - (num_objects-K_plus)* object_dim *np.log(sigma_X) 
    part3 = - (object_dim*K_plus)*np.log(sigma_A) 
    part4 = - (object_dim/2)* np.log(np.dot(np.transpose(Z),Z) + (sigma_X/sigma_A)**2 * np.identity(K_plus))
    part5 = (-1/(2*sigma_X**2)) * np.trace(np.dot(np.dot(np.transpose(X),(np.identity(num_objects) - np.dot(np.dot(Z,M),np.transpose(Z)))),X))
    total = part1+part2+part3+part4+part5
    return(total)

def likelihood1(X, Z, M, sigma_A, sigma_X, K_plus, num_objects, object_dim):
    log_ll = -1 * num_objects * object_dim * .5 * np.log(2*np.pi) - 1 * (num_objects - K_plus) * object_dim * np.log(sigma_X) - K_plus * object_dim * np.log(sigma_A) - object_dim * .5 * np.log(np.linalg.det((np.dot(Z.T, Z) + (sigma_X**2/sigma_A**2)*np.eye(K_plus)))) + (-1/(2*sigma_X**2)) * np.trace(np.dot(np.dot(X.T, np.eye(num_objects) - np.dot(Z, np.dot(M, Z.T))), X))
    return(log_ll)

def likelihood2(X, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim):
    M = np.dot(Z.T, Z) + (sigma_X**2/sigma_A**2) * np.eye(K_plus)
    log_ll = -1 * num_objects * object_dim * .5 * np.log(2*np.pi) - 1 * (num_objects - K_plus) * object_dim * np.log(sigma_X) - K_plus * object_dim * np.log(sigma_A) - object_dim * .5 * np.log(np.linalg.det(M)) + (-1/(2*sigma_X**2)) * np.trace(np.dot(np.dot(X.T, np.eye(num_objects) - np.dot(Z, np.dot(np.linalg.inv(M), Z.T))), X))
    return(log_ll)

In [6]:
def ll(X, Z, sigmaX, sigmaA, K, D, N):
    """Improved log likelihood calculation for X"""
    import numpy as np
    import math
    M = Z.T.dot(Z)+(sigmaX**2/sigmaA**2)*np.identity(K)
    return (-1)*np.log(2*np.pi)*N*D*.5 - np.log(sigmaX)*(N-K)*D - np.log(sigmaA)*K*D - .5*D*np.log(np.linalg.det(M)) \
        -.5/(sigmaX**2)*np.trace( (X.T.dot( np.identity(N)-Z.dot(np.linalg.inv(M).dot(Z.T)) )).dot(X) )

Metropolis Functions
====

In [7]:
from scipy.stats import uniform as unif

def met_sigma_A(Z, K_plus, new_dishes, sigma_X, sigma_A, num_objects, object_dim, X):
    
    #Determine M based on sigma_A
    #Determine current likelihood based on M and sigma_A
    M = Mcalc(Z, K_plus, new_dishes, sigma_X, sigma_A)
    l_curr = likelihood1(X, Z[:,0:(K_plus+new_dishes)], M, sigma_A, sigma_X, K_plus+new_dishes, num_objects, object_dim)
    
    #sample new value from symmetric distribution close to current value
    if unif.rvs(0,1)<0.5:
        new_sigma_A = sigma_A - unif.rvs(0,1)/20
    else:
        new_sigma_A = sigma_A + unif.rvs(0,1)/20
    
    #Determine M based on new_sigma_A
    #Determine current likelihood based on M and new_sigma_A
    M = Mcalc(Z, K_plus, new_dishes, sigma_X, new_sigma_A)
    l_new = likelihood1(x, Z[:,0:(K_plus+new_dishes)], M, new_sigma_A, sigma_X, K_plus+new_dishes, num_objects, object_dim)
    
    acceptance_ratio = np.exp(min(0,l_new - l_curr))
    
    if unif.rvs(0,1)<acceptance_ratio:
        return(new_sigma_A)
    else:
        return(sigma_A)

def met_sigma_X(Z, K_plus, new_dishes, sigma_X, sigma_A, num_objects, object_dim, X):
    
    #Determine M based on sigma_X
    #Determine current likelihood based on M and sigma_X
    M = Mcalc(Z, K_plus, new_dishes, sigma_X, sigma_A)
    l_curr = likelihood1(X, Z[:,0:(K_plus+new_dishes)], M, sigma_A, sigma_X, K_plus+new_dishes, num_objects, object_dim)
    
    #sample new value from symmetric distribution close to current value
    if unif.rvs(0,1)<0.5:
        new_sigma_X = sigma_X - unif.rvs(0,1)/20
    else:
        new_sigma_X = sigma_X + unif.rvs(0,1)/20
    
    #Determine M based on new_sigma_A
    #Determine current likelihood based on M and new_sigma_A
    M = Mcalc(Z, K_plus, new_dishes, new_sigma_X, sigma_A)
    l_new = likelihood1(X, Z[:,0:(K_plus+new_dishes)], M, sigma_A, new_sigma_X, K_plus+new_dishes, num_objects, object_dim)
    
    acceptance_ratio = np.exp(min(0,l_new - l_curr))
    
    if unif.rvs(0,1)<acceptance_ratio:
        return(new_sigma_X)
    else:
        return(sigma_X)

def met_sigma_A1(Z, K_plus, new_dishes, sigma_X, sigma_A, num_objects, object_dim, X):
    
    #Determine M based on sigma_A
    #Determine current likelihood based on M and sigma_A
    #M = Mcalc(Z, K_plus, new_dishes, sigma_X, sigma_A)
    l_curr = likelihood2(X, Z[:,0:(K_plus+new_dishes)], sigma_A, sigma_X, K_plus+new_dishes, num_objects, object_dim)
    
    #sample new value from symmetric distribution close to current value
    if unif.rvs(0,1)<0.5:
        new_sigma_A = sigma_A - unif.rvs(0,1)/20
    else:
        new_sigma_A = sigma_A + unif.rvs(0,1)/20
    
    #Determine M based on new_sigma_A
    #Determine current likelihood based on M and new_sigma_A
    #M = Mcalc(Z, K_plus, new_dishes, sigma_X, new_sigma_A)
    l_new = likelihood2(x, Z[:,0:(K_plus+new_dishes)], new_sigma_A, sigma_X, K_plus+new_dishes, num_objects, object_dim)
    
    acceptance_ratio = np.exp(min(0,l_new - l_curr))
    
    if unif.rvs(0,1)<acceptance_ratio:
        return(new_sigma_A)
    else:
        return(sigma_A)

def met_sigma_X1(Z, K_plus, new_dishes, sigma_X, sigma_A, num_objects, object_dim, X):
    
    #Determine M based on sigma_X
    #Determine current likelihood based on M and sigma_X
    #M = Mcalc(Z, K_plus, new_dishes, sigma_X, sigma_A)
    l_curr = likelihood2(X, Z[:,0:(K_plus+new_dishes)], sigma_A, sigma_X, K_plus+new_dishes, num_objects, object_dim)
    
    #sample new value from symmetric distribution close to current value
    if unif.rvs(0,1)<0.5:
        new_sigma_X = sigma_X - unif.rvs(0,1)/20
    else:
        new_sigma_X = sigma_X + unif.rvs(0,1)/20
    
    #Determine M based on new_sigma_A
    #Determine current likelihood based on M and new_sigma_A
    #M = Mcalc(Z, K_plus, new_dishes, new_sigma_X, sigma_A)
    l_new = likelihood2(X, Z[:,0:(K_plus+new_dishes)], sigma_A, new_sigma_X, K_plus+new_dishes, num_objects, object_dim)
    
    acceptance_ratio = np.exp(min(0,l_new - l_curr))
    
    if unif.rvs(0,1)<acceptance_ratio:
        return(new_sigma_X)
    else:
        return(sigma_X)

Mcalc Function
====

In [8]:
def Mcalc(Z, K_plus, new_dishes, sigma_X, sigma_A):
    part1 = np.dot(np.transpose(Z[:,0:(K_plus+new_dishes)]), Z[:,0:(K_plus+new_dishes)])
    part2 = (sigma_X/sigma_A)**2 * np.identity(K_plus+new_dishes)
    result = np.linalg.inv(part1+part2)
    return(result)


Data Generation
====

In [ ]:
from scipy.stats import multivariate_normal as mvtnorm
from scipy.stats import bernoulli
import numpy as np

### Data Simulation ###

## Features/Latent Variables #
#Each row of the weight matrix "W" specifies one base images(feature/latent variables)"

W = np.array([[0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]])


#Each image in our simulated data set is the superposition of four base images#
# Number of images/ data points
num_objects=100

#Dimension of image (6x6)
object_dim  = 6*6

#Covariance matrix for images/ white noise
sigma_x_orig = 0.5
I = sigma_x_orig * np.identity(object_dim)

#z_i - binary feature matrix (1 x 4) - each entry set to 1 with probability 0.5 and 0 otherwise#
#x is data variable - each row correspondes to a superimposed built from a random combination of latent features#
#with white noise added - x is built with multivariate gaussian#
x = np.zeros((100,36))
z = np.zeros((100,4))

for i in range(0,num_objects):
    z[i,:] = np.array([bernoulli.rvs(p=0.5, size=4)])
    x[i,:] = mvtnorm.rvs(np.dot(z[i,:],W), I)

The Sampler
====

In [ ]:
# GENERAL FUNCTION SAMPLER

# Harmonic number or N
HN = 0
for i in range(0, num_objects):
    HN = HN + 1/(i+1)

E = 1000
BURN_IN = 0
SAMPLE_SIZE = 1000

# Initializing values for use in our chain
sigma_A = 1
sigma_X = 1
# Poisson rate
alpha = 1
# Prespecified maximum number of latent features
K_inf = 20
# Indian Buffet Process Prior
Z, K_plus = sampleIBP(alpha, num_objects)
# Initialization of our chain for Z
chain_Z = np.zeros([SAMPLE_SIZE, num_objects, K_inf])
# Initialization of our chain for K
chain_K = np.zeros([SAMPLE_SIZE, 1])
# Initialization of our chain for sigma_X
chain_sigma_X = np.zeros([SAMPLE_SIZE, 1])
# Initialization of our chain for sigma_A
chain_sigma_A = np.zeros([SAMPLE_SIZE, 1])
# Initialization of our chain for alpha
chain_alpha = np.zeros([SAMPLE_SIZE, 1])

# Initializing storage for post burn-in samples
s_counter = 0
for e in range(0, E):
    if(e > BURN_IN):
        s_counter = s_counter + 1
        chain_Z[s_counter, :, 0:K_plus] = Z[:, 0:K_plus]
        chain_K[s_counter] = K_plus
        chain_sigma_X[s_counter] = sigma_X
        chain_sigma_A[s_counter] = sigma_A
        chain_alpha[s_counter] = alpha
    print("At iteration", e, ": K_plus is", K_plus, ", alpha is", alpha) 

    for i in range(0, num_objects):
        # M matrix will be handy for future computations
        # SOMETIMES SINGULAR, NEED TO FIX
        #M = np.linalg.inv(np.dot(Z[:, 0:K_plus].T, Z[:, 0:K_plus]) + (sigma_X**2/sigma_A**2) * np.eye(K_plus))
        for k in range(0, K_plus):
            # Checking to make sure that k < K_plus
            if k >= K_plus:
                break
            if Z[i, k] > 0:
                # Take care of singularities
                if (np.sum(Z[:, k]) - Z[i, k]) <= 0: 
                    Z[i, k] = 0
                    Z[:, k:(K_plus - 1)] = Z[:, (k+1):K_plus]
                    K_plus = K_plus - 1
                    Z = Z[:, 0:K_plus]
                    #M = np.linalg.inv(np.dot(Z[:, 0:K_plus].T, Z[:, 0:K_plus]) + (sigma_X**2/sigma_A**2) * np.eye(K_plus))
                    continue
             
            # This is where he has his calcInverse functions to 
            # speed up inverse calculations. I just use inverse
            # of Z matrix below
            #M1 = calcInverse(Z[:, 0:K_plus], M, i, k, 1)
            #M2 = calcInverse(Z[:, 0:K_plus], M, i, k, 0)

            # Compute conditional distributions for the current cell in Z
            P = np.zeros(2)
            Z[i, k] = 1
            #P[0] = likelihood1(x, Z[:, 0:K_plus], M1, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(np.sum(Z[:, k]) - 1) - np.log(num_objects)
            P[0] = likelihood2(x, Z[:, 0:K_plus], sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(np.sum(Z[:, k]) - 1) - np.log(num_objects)

            Z[i, k] = 0
            #P[1] = likelihood1(x, Z[:, 0:K_plus], M2, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(num_objects - np.sum(Z[:,k])) - np.log(num_objects)
            P[1] = likelihood2(x, Z[:, 0:K_plus], sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(num_objects - np.sum(Z[:,k])) - np.log(num_objects)
            
            P = np.exp(P - np.max(P))

            # Sample from the conditional
            if np.random.uniform() < (P[0] / (np.sum(P))):
                Z[i, k] = 1
                #M = M1
            else:
                Z[i, k] = 0
                #M = M2
        
        # Sample the number of new dishes for the current object
        trun = np.zeros([1, 5])
        alpha_N = alpha / num_objects


################### ERROR IN DIMENSIONALITY RIGHT HERE ###########################        
        for k_i in range(5):
            Z_temp = Z        
            if k_i > 0:
                z = np.zeros((num_objects, k_i))
                Z_temp = np.column_stack((Z_temp, z))
                Z_temp[i, K_plus:(K_plus+k_i)] = 1
            #M = np.linalg.inv(np.dot(Z_temp[:, 0:(K_plus + k_i)].T, Z_temp[:, 0:(K_plus+k_i)]) + (sigma_X**2/sigma_A**2) * np.eye(K_plus + k_i)) # MINUS ONE TO FIX DIMENSIONALITY MAY BE INCORRECT
            #trun[0, k_i] = k_i * np.log(alpha_N) - alpha_N - np.log(np.math.factorial(k_i)) + likelihood1(x, Z_temp[:, 0:(K_plus + k_i)], M, sigma_A, sigma_X, K_plus + k_i, num_objects, object_dim)
            trun[0, k_i] = k_i * np.log(alpha_N) - alpha_N - np.log(np.math.factorial(k_i)) + likelihood2(x, Z_temp[:, 0:(K_plus + k_i)], sigma_A, sigma_X, K_plus + k_i, num_objects, object_dim)
        #Z[i, K_plus:(K_plus+4)] = 0
        trun = np.exp(trun - np.max(trun))
        trun = trun/np.sum(trun)
        p = np.random.uniform()
        t = 0
        new_dishes = 0
        for k_i in range(0,5):
            t = t + trun[0, k_i]
            if p < t:
                new_dishes = k_i
                break
        if(new_dishes > 0):
            z = np.zeros((num_objects, new_dishes))
            Z = np.column_stack([Z, z])
            Z[i, K_plus:(K_plus + new_dishes)] = 1
        K_plus = K_plus + new_dishes
    
    # Metropolis steps for sampling sigma_X and sigma_A
    likelihood_current = likelihood2(x, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim)
    
    if np.random.uniform(0,1) < .5:
        sigma_X_new = sigma_X - np.random.uniform(0,1)/20
    else:
        sigma_X_new = sigma_X + np.random.uniform(0,1)/20

    likelihood_new = likelihood2(x, Z, sigma_A, sigma_X_new, K_plus, num_objects, object_dim)
    acc_X = np.exp(min(0, likelihood_new - likelihood_current))

    if np.random.uniform(0,1) < .5:
        sigma_A_new = sigma_A - np.random.uniform(0,1)/20
    else:
        sigma_A_new = sigma_A + np.random.uniform(0,1)/20
    
    likelihood_new = likelihood2(x, Z, sigma_A_new, sigma_X, K_plus, num_objects, object_dim)
    acc_A = np.exp(min(0, likelihood_new - likelihood_current))

    if np.random.uniform(0,1) < acc_X:
        sigma_X = sigma_X_new
    if np.random.uniform(0,1) < acc_A:
        sigma_A = sigma_A_new
   

    #N_sigma_X = met_sigma_X1(Z, K_plus, new_dishes, sigma_X, sigma_A, num_objects, object_dim, x)

    #sigma_A = met_sigma_A1(Z, K_plus, new_dishes, sigma_X, sigma_A, num_objects, object_dim, x)
    
    #sigma_X = N_sigma_X

    # Sample alpha from its conditional posterior
    alpha = np.random.gamma(1 + K_plus, 1/(1+HN))


At iteration 0 : K_plus is 5 , alpha is 1
At iteration 1 : K_plus is 4 , alpha is 1.3963298378211284
At iteration 2 : K_plus is 2 , alpha is 0.6169403505179917
At iteration 3 : K_plus is 2 , alpha is 0.23767865160230417
At iteration 4 : K_plus is 1 , alpha is 0.7427434824493387
At iteration 5 : K_plus is 1 , alpha is 0.5295302638421566
At iteration 6 : K_plus is 1 , alpha is 1.0437120795599208
At iteration 7 : K_plus is 2 , alpha is 0.6731305353864493
At iteration 8 : K_plus is 2 , alpha is 0.8003158668776872
At iteration 9 : K_plus is 3 , alpha is 0.5608862853738201
At iteration 10 : K_plus is 3 , alpha is 0.3115594150053396
At iteration 11 : K_plus is 3 , alpha is 0.7312325926511918
At iteration 12 : K_plus is 3 , alpha is 0.24199112464864006
At iteration 13 : K_plus is 2 , alpha is 0.23761165011956945
At iteration 14 : K_plus is 3 , alpha is 0.8536931636391178
At iteration 15 : K_plus is 3 , alpha is 0.2550275984592406
At iteration 16 : K_plus is 2 , alpha is 0.28564608099693
At ite

In [33]:
Z = Z[:,0:(K_plus+new_dishes)]
K_plus = K_plus+new_dishes
K_plus
#np.dot(Z.T, Z) + (sigma_X**2/sigma_A**2) * np.eye(K_plus)

16

In [16]:
for i in range(10):
    for k in range(10):
        

(array([], dtype=float64), 0)

In [54]:
def profiler(n):
    # GENERAL FUNCTION SAMPLER

    # Harmonic number or N
    HN = 0
    for i in range(0, num_objects):
        HN = HN + 1/(i+1)

    E = n
    BURN_IN = 0
    SAMPLE_SIZE = n

    # Initializing values for use in our chain
    sigma_A = 1
    sigma_X = 1
    # Poisson rate
    alpha = 1
    # Prespecified maximum number of latent features
    K_inf = 10
    # Indian Buffet Process Prior
    Z, K_plus = sampleIBP(alpha, num_objects)
    # Initialization of our chain for Z
    chain_Z = np.zeros([SAMPLE_SIZE, num_objects, K_inf])
    # Initialization of our chain for K
    chain_K = np.zeros([SAMPLE_SIZE, 1])
    # Initialization of our chain for sigma_X
    chain_sigma_X = np.zeros([SAMPLE_SIZE, 1])
    # Initialization of our chain for sigma_A
    chain_sigma_A = np.zeros([SAMPLE_SIZE, 1])
    # Initialization of our chain for alpha
    chain_alpha = np.zeros([SAMPLE_SIZE, 1])

    # Initializing storage for post burn-in samples
    s_counter = 0
    for e in range(0, E):
        if(e > BURN_IN):
            s_counter = s_counter + 1
            chain_Z[s_counter, :, 0:K_plus] = Z[:, 0:K_plus]
            chain_K[s_counter] = K_plus
            chain_sigma_X[s_counter] = sigma_X
            chain_sigma_A[s_counter] = sigma_A
            chain_alpha[s_counter] = alpha
        print("At iteration", e, ": K_plus is", K_plus, ", alpha is", alpha) 

        for i in range(0, num_objects):
            # M matrix will be handy for future computations
            # SOMETIMES SINGULAR, NEED TO FIX
            M = np.linalg.inv(np.dot(Z[:, 0:K_plus].T, Z[:, 0:K_plus]) + (sigma_X**2/sigma_A**2) * np.eye(K_plus))
            for k in range(0, K_plus):
                # Checking to make sure that k < K_plus
                if k+1 > K_plus:
                    break
                if Z[i, k] > 0:
                    # Take care of singularities
                    if np.sum(Z[:, k]) - Z[i, k] <= 0:
                        Z[i, k] = 0
                        Z[:, k:(K_plus - 1)] = Z[:, (k+1):K_plus]
                        K_plus = K_plus - 1
                        M = np.linalg.inv(np.dot(Z[:, 0:K_plus].T, Z[:, 0:K_plus]) + (sigma_X**2/sigma_A**2) * np.eye(K_plus))
                        continue
             
                # This is where he has his calcInverse functions to 
                # speed up inverse calculations. I just use inverse
                # of Z matrix below
                M1 = calcInverse(Z[:, 0:K_plus], M, i, k, 1)
                M2 = calcInverse(Z[:, 0:K_plus], M, i, k, 0)

                # Compute conditional distributions for the current cell in Z
                P = []
                Z[i, k] = 1
                P.append(likelihood1(x, Z[:, 0:K_plus], M1, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(sum(Z[:, k]) - Z[i, k]) - np.log(num_objects))

                Z[i, k] = 0
                P.append(likelihood1(x, Z[:, 0:K_plus], M2, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(num_objects - np.sum(Z[:,k])) - np.log(num_objects))
            
                P = np.exp(P - np.max(P))

                # Sample from the conditional
                if np.random.uniform() < P[0] / (P[0] + P[1]):
                    Z[i, k] = 1
                    M = M1
                else:
                    Z[i, k] = 0
                    M = M2
        
            # Sample the number of new dishes for the current object
            trun = np.zeros([1, 5])
            alpha_N = alpha / num_objects


################### ERROR IN DIMENSIONALITY RIGHT HERE ###########################        
            for k_i in range(0, 5):
                if k_i > 0:
                    z = np.repeat(0, np.shape(Z)[0])
                    Z = np.column_stack((Z, z))
                    Z[i, K_plus:(K_plus+k_i)] = 1
                M = np.linalg.inv(np.dot(Z[:, 0:(K_plus + k_i)].T, Z[:, 0:(K_plus+k_i)]) + (sigma_X**2/sigma_A**2) * np.eye(K_plus + k_i)) # MINUS ONE TO FIX DIMENSIONALITY MAY BE INCORRECT
                trun[0, k_i] = k_i * np.log(alpha_N) - alpha_N - np.log(np.math.factorial(k_i)) + likelihood1(x, Z[:, 0:(K_plus + k_i)], M, sigma_A, sigma_X, K_plus + k_i, num_objects, object_dim)
            Z[i, K_plus:(K_plus+4)] = 0
            trun = np.exp(trun - np.max(trun))
            trun = trun/np.sum(trun)
            p = np.random.uniform()
            t = 0
            for k_i in range(0,5):
                t = trun[0, k_i]
                if p < t:
                    new_dishes = k_i
                    break
            Z[i, K_plus:(K_plus + new_dishes)] = 1
            K_plus = K_plus + new_dishes
    
        # Metropolis steps for sampling sigma_X and sigma_A
        sigma_X = met_sigma_X(Z, K_plus, new_dishes, sigma_X, sigma_A, num_objects, object_dim, x)

        sigma_A = met_sigma_A(Z, K_plus, new_dishes, sigma_X, sigma_A, num_objects, object_dim, x)

        # Sample alpha from its conditional posterior
        alpha = gamma.rvs(a = 1 + K_plus, scale = 1/(1+HN))

In [ ]:
%prun -q -D profiler.prof profiler(int(1))

In [ ]:
import pstats
p = pstats.Stats('profiler.prof')
p.print_stats()
pass